In [28]:
import os
import re
import io
import logging
import requests
import pandas as pd
import fitz
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [45]:
# --- КОНФИГУРАЦИЯ ---
# Настройка логирования
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

class STANKIN_RAG_Config:
    START_URL = "https://priem.stankin.ru/"
    BASE_DOMAIN = urlparse(START_URL).netloc
    MAX_CRAWL_DEPTH = 4
    EMBEDDING_MODEL_NAME = "cointegrated/rubert-tiny2"
    CHROMA_DB_PATH = "./stankin_db"
    COLLECTION_NAME = "stankin_collection"
    
    # Параметры чанкинга
    CHUNK_SIZE = 500  # Используем больше, но фильтр по длине все равно сработает
    CHUNK_OVERLAP = 150
    MIN_CHUNK_LEN = 50
    MAX_CHUNK_LEN = 3000
    
    # Регулярные выражения для Анти-Мусор Фильтра (Требование 2.C)
    ANTI_GARBAGE_PATTERNS = [
        re.compile(r'data:image/[a-zA-Z0-9+/=;,-]+'),  # Base64 строки
        re.compile(r'', re.DOTALL),          # HTML комментарии
        re.compile(r'\[Top\.Mail\.Ru\]|Yandex\.Metrika'), # Счетчики/Аналитика
        re.compile(r'^\s*https?://\S+\s*$')              # Чанк - это просто URL
    ]

    PDF_DOCUMENTS = [
        "https://stankin.ru/uploads/files/file_60fa90522da0f.pdf",
        "https://newcloud.stankin.ru/s/sk5ermw7k9Kdxxs/download/2025_%D0%9F%D0%BE%D1%80%D1%8F%D0%B4%D0%BE%D0%BA_%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D1%8F_%D0%B2%D1%81%D1%82%D1%83%D0%BF%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D1%85_%D0%B8%D1%81%D0%BF%D1%8B%D1%82%D0%B0%D0%BD%D0%B8%D0%B9_%D0%B2_%D0%B0%D1%81%D0%BF%D0%B8%D1%80%D0%B0%D0%BD%D1%82%D1%83%D1%80%D1%83.pdf",
        "https://newcloud.stankin.ru/s/3X3GL68qHBRZnKN/download/2025_%D0%9F%D0%BE%D1%80%D1%8F%D0%B4%D0%BE%D0%BA_%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D1%8F_%D0%B2%D1%81%D1%82%D1%83%D0%BF%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D1%85_%D0%B8%D1%81%D0%BF%D1%8B%D1%82%D0%B0%D0%BD%D0%B8%D0%B9_%D0%B2_%D0%B1%D0%B0%D0%BA,_%D1%81%D0%BF%D0%B5%D1%86.pdf",
        "https://newcloud.stankin.ru/s/sndBbRQqy6wRLCz/download/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%BF%D1%80%D0%B8%D0%B5%D0%BC%D0%B0_2025_%D0%90%D0%A1%D0%9F.pdf",
        "https://newcloud.stankin.ru/s/YozjQrwrYcGeDF3/download/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%BF%D1%80%D0%B8%D1%91%D0%BC%D0%B0_2025_%D0%91%D0%A1%D0%9C.pdf"
        
        # Добавьте сюда ваши конкретные PDF-файлы
    ]

In [46]:
# --- МОДУЛЬ 1: ВЕКТОРИЗАЦИЯ (ОБНОВЛЕННЫЙ) ---
class SBERT_EmbeddingFunction(EmbeddingFunction):
    """
    Использует SentenceTransformer для надежной загрузки DeepPavlov модели,
    гарантируя корректный пулинг и нормализацию.
    """
    def __init__(self, model_name: str):
        logging.info(f"Загрузка модели через SentenceTransformer: {model_name}...")
        try:
            # SentenceTransformer автоматически обрабатывает pooling и нормализацию
            self.model = SentenceTransformer(model_name)
            logging.info("SUCCESS: Функция эмбеддинга (SBERT) успешно инициализирована.")
        except Exception as e:
            logging.critical(f"КРИТИЧЕСКАЯ ОШИБКА ЗАГРУЗКИ SBERT: {e}")
            raise

    def __call__(self, texts: Documents) -> Embeddings:
        """Генерирует эмбеддинги."""
        # encode возвращает L2-нормализованные векторы numpy
        embeddings = self.model.encode(texts, convert_to_numpy=True)
        return embeddings.tolist()

In [47]:
# --- МОДУЛЬ 2: КРАУЛИНГ И ОЧИСТКА ---

def clean_html_content(html_content: str, url: str) -> str:
    """
    Агрессивная очистка HTML: удаление служебных тегов, 
    конвертация таблиц в Markdown.
    """
    soup = BeautifulSoup(html_content, 'lxml')
    
    # 1. Удаление служебных тегов
    REMOVAL_TAGS = ['script', 'style', 'nav', 'footer', 'header', 'form', 'aside', 'iframe', 'noscript']
    for tag in REMOVAL_TAGS:
        for element in soup.find_all(tag):
            element.decompose()
    logging.debug(f"Удалены служебные теги на {url}")

    # 2. Обработка Таблиц (Преобразование в Markdown)
    tables = soup.find_all('table')
    if tables:
        logging.info(f"Обработка {len(tables)} таблиц на странице {url}...")
        for i, table in enumerate(tables):
            try:
                # pandas.read_html может парсить как строку, так и тег
                df = pd.read_html(str(table))[0]
                # Конвертация в Markdown
                markdown_table = "\n\n--- НАЧАЛО ТАБЛИЦЫ ---\n"
                markdown_table += df.to_markdown(index=False)
                markdown_table += "\n--- КОНЕЦ ТАБЛИЦЫ ---\n\n"
                
                # Заменяем тег <table> на его Markdown представление
                table.replace_with(BeautifulSoup(markdown_table, 'html.parser'))
            except Exception as e:
                logging.warning(f"Ошибка парсинга таблицы {i+1} на {url}: {e}")
                table.decompose() # Удаляем проблемную таблицу

    # 3. Финальная очистка текста
    # Получаем чистый текст из оставшегося HTML
    text = soup.get_text(separator=' ', strip=True)
    # Удаляем лишние пробелы и переводы строк
    text = re.sub(r'\s+', ' ', text).strip()
    
    logging.debug(f"HTML-контент страницы {url} очищен. Длина: {len(text)}")
    return text

In [48]:
def stankin_crawler(start_url: str, max_depth: int) -> dict:
    """
    Рекурсивный краулинг с ограничением по домену и глубине.
    Возвращает словарь {url: content}.
    """
    queue = [(start_url, 0)]
    visited = {start_url}
    page_contents = {}
    base_domain = STANKIN_RAG_Config.BASE_DOMAIN
    
    logging.info(f"Запуск краулинга с {start_url} до глубины {max_depth}...")

    while queue:
        current_url, depth = queue.pop(0)

        if depth > max_depth:
            continue
        
        logging.info(f"-> Парсинг URL: {current_url} (Глубина: {depth})")

        try:
            response = requests.get(current_url, timeout=10)
            # Проверяем успешность и кодировку
            response.raise_for_status()
            response.encoding = response.apparent_encoding 
            
            html_content = response.text
            
            # 1. Очищаем контент
            cleaned_text = clean_html_content(html_content, current_url)
            page_contents[current_url] = cleaned_text
            
            # 2. Поиск новых ссылок
            soup = BeautifulSoup(html_content, 'lxml')
            for link in soup.find_all('a', href=True):
                href = link['href']
                absolute_url = urljoin(current_url, href).split('#')[0] # Убираем якоря
                
                # Проверка на поддомен и посещение
                if urlparse(absolute_url).netloc == base_domain and absolute_url not in visited:
                    visited.add(absolute_url)
                    queue.append((absolute_url, depth + 1))
                    logging.debug(f"DEBUG: Найдена новая ссылка: {absolute_url} (Depth: {depth + 1})")
            
        except requests.exceptions.RequestException as e:
            logging.error(f"Ошибка при загрузке {current_url}: {e}")
        except Exception as e:
            logging.error(f"Непредвиденная ошибка на {current_url}: {e}")

    logging.info(f"SUCCESS: Краулинг завершен. Найдено {len(page_contents)} уникальных страниц.")
    return page_contents

In [49]:
# --- МОДУЛЬ 3: ЧАНКИНГ И ФИЛЬТРАЦИЯ ---

def create_and_filter_chunks(page_contents: dict) -> list[dict]:
    """
    Разбивает текст на чанки и применяет двойную фильтрацию.
    Возвращает список словарей {text, source}.
    """
    # Инициализация LangChain Text Splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=STANKIN_RAG_Config.CHUNK_SIZE,
        chunk_overlap=STANKIN_RAG_Config.CHUNK_OVERLAP,
        separators=["\n\n", "\n", " ", ""],
        length_function=len
    )
    
    final_chunks = []
    
    logging.info("Применение чанкинга и двойного фильтра ко всему контенту...")

    for url, text in page_contents.items():
        if not text:
            logging.warning(f"Пропущен пустой контент для URL: {url}")
            continue

        # 1. Разбиение на чанки
        chunks = text_splitter.split_text(text)
        logging.info(f"URL: {url} -> Исходный текст разбит на {len(chunks)} чанков.")
        
        for i, chunk in enumerate(chunks):
            # 2. Фильтр 1: По длине (Требование 2.C)
            if not (STANKIN_RAG_Config.MIN_CHUNK_LEN <= len(chunk) <= STANKIN_RAG_Config.MAX_CHUNK_LEN):
                logging.debug(f"CHUNK FILTER (Length): Пропущен чанк #{i} (Длина: {len(chunk)}).")
                continue

            '''
            # 3. Фильтр 2: Анти-Мусор (Требование 2.C)
            is_garbage = False
            for pattern in STANKIN_RAG_Config.ANTI_GARBAGE_PATTERNS:
                if pattern.search(chunk):
                    logging.warning(f"CHUNK FILTER (Garbage): Пропущен чанк #{i} из {url} (Начало: {chunk[:50]}...)")
                    is_garbage = True
                    break
            
            if is_garbage:
                continue
            '''

            # Если чанк прошел все фильтры
            final_chunks.append({
                "text": chunk,
                "source": url
            })

    logging.info(f"SUCCESS: Итоговое количество чанков, готовых к индексированию: {len(final_chunks)}")
    return final_chunks

In [50]:
# --- МОДУЛЬ 4: ИНДЕКСИРОВАНИЕ ---

def index_chunks_to_chroma(chunks: list[dict], ef: EmbeddingFunction):
    """
    Индексирует очищенные чанки в ChromaDB.
    """
    logging.info(f"Инициализация ChromaDB по пути: {STANKIN_RAG_Config.CHROMA_DB_PATH}")
    client = chromadb.PersistentClient(path=STANKIN_RAG_Config.CHROMA_DB_PATH)
    collection_name = STANKIN_RAG_Config.COLLECTION_NAME
    
    # Требование 2.D: Удаление существующей коллекции
    try:
        client.delete_collection(name=collection_name)
        logging.critical(f"Удалена старая коллекция '{collection_name}' для чистого старта.")
    except:
        logging.info(f"Коллекция '{collection_name}' не найдена или не требует удаления.")

    # Создание новой коллекции с кастомной функцией эмбеддинга
    collection = client.get_or_create_collection(
        name=collection_name, 
        embedding_function=ef,
        metadata={"hnsw:space": "cosine"}
    )
    logging.info(f"Коллекция '{collection_name}' готова к заполнению.")

    # Подготовка данных для пакетного добавления
    documents = [c['text'] for c in chunks]
    metadatas = [{"source": c['source']} for c in chunks]
    # ID's должны быть уникальными (например, URL + индекс чанка)
    ids = [f"{i}-{re.sub(r'[^a-zA-Z0-9]', '_', c['source'])}" for i, c in enumerate(chunks)]

    if not documents:
        logging.warning("Нет документов для индексирования. Пропускаем.")
        return

    # Пакетное добавление
    BATCH_SIZE = 500 
    for i in range(0, len(documents), BATCH_SIZE):
        batch_docs = documents[i:i + BATCH_SIZE]
        batch_metadatas = metadatas[i:i + BATCH_SIZE]
        batch_ids = ids[i:i + BATCH_SIZE]
        
        collection.add(
            documents=batch_docs,
            metadatas=batch_metadatas,
            ids=batch_ids
        )
        logging.info(f"Индексирован пакет: {i} - {min(i + BATCH_SIZE, len(documents))}")
    
    logging.critical(f"SUCCESS: Индексирование завершено. Всего чанков в БД: {collection.count()}")

In [51]:
# --- МОДУЛЬ 5: ТЕСТИРОВАНИЕ RAG ---

def test_rag_query(query: str, ef: EmbeddingFunction):
    """
    Выполняет тестовый запрос и выводит результаты.
    """
    logging.info("-" * 50)
    logging.info("НАЧАЛО ТЕСТИРОВАНИЯ КАЧЕСТВА RAG")
    logging.info(f"Тестовый запрос: '{query}'")
    
    client = chromadb.PersistentClient(path=STANKIN_RAG_Config.CHROMA_DB_PATH)
    collection_name = STANKIN_RAG_Config.COLLECTION_NAME
    
    try:
        # Получаем коллекцию (с обязательным указанием EF, чтобы запрос кодировался корректно)
        collection = client.get_collection(name=collection_name, embedding_function=ef)
    except Exception as e:
        logging.error(f"Коллекция не найдена. Невозможно выполнить тест. {e}")
        return

    # Выполняем семантический поиск
    results = collection.query(
        query_texts=[query],
        n_results=5, # Требование 2.D
        include=['documents', 'metadatas', 'distances']
    )

    if not results or not results['documents'] or not results['documents'][0]:
        logging.error("Не найдено релевантных документов.")
        return

    logging.critical("\n--- 5 САМЫХ РЕЛЕВАНТНЫХ ЧАНКОВ (RAG-РЕЗУЛЬТАТ) ---\n")
    
    for i in range(5):
        try:
            distance = results['distances'][0][i]
            source = results['metadatas'][0][i]['source']
            content = results['documents'][0][i]
            
            print(f"[{i+1}] Дистанция: {distance:.4f}")
            print(f"    Источник: {source}")
            print(f"    Содержание: {content[:300]}...\n")
            
        except IndexError:
            # Если найдено меньше 5 результатов
            break
            
    logging.info("-" * 50)

In [52]:
# --- МОДУЛЬ: PDF-ПАРСИНГ ---
def extract_text_from_pdf_url(pdf_url: str) -> str | None:
    """
    Скачивает PDF по URL и извлекает из него текст постранично, 
    используя контекстный менеджер для надежного закрытия.
    """
    logging.info(f"Начало обработки PDF: {pdf_url}")
    
    try:
        # 1. Скачивание содержимого PDF
        response = requests.get(pdf_url, timeout=20) # Увеличим таймаут на всякий случай
        response.raise_for_status() 
        pdf_content = response.content
        
        # 2. Использование fitz.open через BytesIO
        pdf_stream = io.BytesIO(pdf_content)
        
        # 3. Открытие документа и использование try/finally для надежного закрытия
        # Мы используем fitz.open() в try-блоке и закрываем в finally-блоке.
        pdf_document = fitz.open(stream=pdf_stream, filetype="pdf")
        
        full_text = []
        try:
            # 4. Извлечение текста
            for page_num in range(len(pdf_document)):
                page = pdf_document.load_page(page_num)
                text = page.get_text("text")
                full_text.append(f"\n--- Страница {page_num + 1} ---\n{text}")
            
            logging.info(f"SUCCESS: Извлечено текста из PDF ({len(pdf_document)} страниц): {pdf_url}")
            return "\n".join(full_text)
            
        finally:
            # pdf_document.close() гарантированно вызывается здесь,
            # и только после того, как все чтение завершено.
            pdf_document.close()
            
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка HTTP при загрузке PDF {pdf_url}: {e}")
        return None
    except Exception as e:
        # Сюда попадет и та самая ошибка 'document closed'
        logging.error(f"Ошибка парсинга PDF {pdf_url}: {e}")
        return None

# --- НОВАЯ ФУНКЦИЯ: ОБРАБОТКА СПИСКА PDF ---
def process_pdf_documents(pdf_urls: list[str]) -> dict[str, str]:
    """
    Парсит список URL-адресов PDF и возвращает словарь: {URL: Текст}.
    """
    pdf_contents = {}
    for url in pdf_urls:
        text = extract_text_from_pdf_url(url)
        if text:
            pdf_contents[url] = text
    return pdf_contents

In [53]:
def index_data_if_needed(ef: SBERT_EmbeddingFunction) -> bool:
    """
    Выполняет полный пайплайн индексации, если ChromaDB пуста, 
    или если коллекция требует обновления (например, удаляем и создаем заново).
    Возвращает True, если индексация прошла успешно.
    """
    logging.info("ПРОВЕРКА: Проверяем, нужна ли переиндексация...")

    # Здесь можно добавить логику проверки:
    # 1. Если файл базы данных не существует, возвращаем True
    # 2. Если вы хотите всегда пересоздавать базу, просто продолжаем.

    # -------------------------------------------------------------------
    # 1. Сбор и Чанкинг HTML-данных
    logging.info("--- ЭТАП 1: СБОР HTML ДАННЫХ ---")
    html_page_contents = stankin_crawler(
        start_url=STANKIN_RAG_Config.START_URL,
        max_depth=STANKIN_RAG_Config.MAX_CRAWL_DEPTH
    )
    html_chunks = create_and_filter_chunks(html_page_contents)
    logging.info(f"Итого HTML чанков после фильтрации: {len(html_chunks)}")

    # -------------------------------------------------------------------
    # 2. Сбор и Чанкинг PDF-данных
    logging.info("--- ЭТАП 2: СБОР PDF ДАННЫХ ---")
    # Используем Ваше исправленное извлечение PDF
    pdf_page_contents = process_pdf_documents(STANKIN_RAG_Config.PDF_DOCUMENTS)
    pdf_chunks = create_and_filter_chunks(pdf_page_contents)
    logging.info(f"Итого PDF чанков после фильтрации: {len(pdf_chunks)}")

    # -------------------------------------------------------------------
    # 3. Объединение и Индексация
    logging.info("--- ЭТАП 3: ОБЪЕДИНЕНИЕ И ИНДЕКСАЦИЯ ---")
    all_chunks = html_chunks + pdf_chunks 

    if all_chunks:
        # Индексируем, что включает удаление старой коллекции
        index_chunks_to_chroma(all_chunks, ef)
        logging.critical(f"SUCCESS: Общее количество проиндексированных чанков: {len(all_chunks)}")
        return True
    else:
        logging.critical("СИСТЕМА ПУСТА: Нет данных для индексирования.")
        return False

In [54]:
# logging.critical("СТАРТ RAG-СИСТЕМЫ ПАРСИНГА СТАНКИНА")

# # -------------------------------------------------------------------
# # ШАГ 1: Инициализация Embedding Function
# # -------------------------------------------------------------------
# try:
#     # ИСПОЛЬЗУЕМ КЛАСС SBERT_EmbeddingFunction
#     ef = SBERT_EmbeddingFunction(STANKIN_RAG_Config.EMBEDDING_MODEL_NAME)
# except Exception as e:
#     logging.critical(f"КРИТИЧЕСКАЯ ОШИБКА ИНИЦИАЛИЗАЦИИ МОДЕЛИ: {e}")

# # Инициализация для предотвращения NameError
# html_chunks = []
# pdf_chunks = []

# # -------------------------------------------------------------------
# # ШАГ 2: Сбор, Очистка и Чанкинг HTML-данных
# # -------------------------------------------------------------------
# logging.info("--- ЭТАП 1: СБОР HTML ДАННЫХ ---")
# try:
#     page_contents = stankin_crawler(
#         start_url=STANKIN_RAG_Config.START_URL,
#         max_depth=STANKIN_RAG_Config.MAX_CRAWL_DEPTH
#     )
#     # *** ИСПРАВЛЕНИЕ: Определение html_chunks ***
#     html_chunks = create_and_filter_chunks(page_contents) 
#     logging.info(f"Итого HTML чанков после фильтрации: {len(html_chunks)}")
# except Exception as e:
#     logging.error(f"Ошибка в процессе HTML-краулинга: {e}")

# # -------------------------------------------------------------------
# # ШАГ 3: Сбор, Очистка и Чанкинг PDF-данных
# # -------------------------------------------------------------------
# logging.info("--- ЭТАП 2: СБОР PDF ДАННЫХ ---")
# try:
#     pdf_page_contents = process_pdf_documents(STANKIN_RAG_Config.PDF_DOCUMENTS)
#     pdf_chunks = create_and_filter_chunks(pdf_page_contents)
#     logging.info(f"Итого PDF чанков после фильтрации: {len(pdf_chunks)}")
# except Exception as e:
#     logging.error(f"Ошибка в процессе парсинга PDF: {e}")
    
# # -------------------------------------------------------------------
# # ШАГ 4: Объединение и Индексация в ChromaDB
# # -------------------------------------------------------------------
# logging.info("--- ЭТАП 3: ОБЪЕДИНЕНИЕ И ИНДЕКСАЦИЯ ---")

# # Объединяем оба списка чанков
# all_chunks = html_chunks + pdf_chunks 

# if all_chunks:
#     # Индексируем ВСЕ чанки в одну коллекцию
#     index_chunks_to_chroma(all_chunks, ef)
#     logging.critical(f"SUCCESS: Общее количество проиндексированных чанков: {len(all_chunks)}")
# else:
#     logging.critical("СИСТЕМА ПУСТА: Нет данных для индексирования.")

# # -------------------------------------------------------------------
# # ШАГ 5: Тестирование RAG
# # -------------------------------------------------------------------
# logging.info("--- ЭТАП 4: ТЕСТИРОВАНИЕ RAG-ЗАПРОСОВ ---")
# TEST_QUERY_1 = "Какие документы нужны для подачи заявления в МГТУ СТАНКИН?"
# TEST_QUERY_2 = "Сколько баллов нужно набрать, для того чтобы поступить на направление 09.03.03?"
# TEST_QUERY_3 = "Сколько баллов дают за индивидуальные достижения?"
# TEST_QUERY_4 = "Какие направления подготовки бакалавриата есть в Станкине?"
# TEST_QUERY_5 = "Когда начинаются вступительные испытания в магистратуре?"

# test_rag_query(TEST_QUERY_1, ef)
# test_rag_query(TEST_QUERY_2, ef)
# test_rag_query(TEST_QUERY_3, ef)
# test_rag_query(TEST_QUERY_4, ef)
# test_rag_query(TEST_QUERY_5, ef)

In [55]:
def main():
    logging.critical("СТАРТ RAG-СИСТЕМЫ: ГИБРИДНЫЙ РЕЖИМ")
    
    # 1. Инициализация модели (нужна и для индексации, и для запросов)
    try:
        ef = SBERT_EmbeddingFunction(STANKIN_RAG_Config.EMBEDDING_MODEL_NAME)
    except Exception as e:
        logging.critical(f"КРИТИЧЕСКАЯ ОШИБКА ИНИЦИАЛИЗАЦИИ МОДЕЛИ: {e}")
        return

In [56]:
index_data_if_needed(ef)

2025-12-14 16:57:14,918 - INFO - ПРОВЕРКА: Проверяем, нужна ли переиндексация...
2025-12-14 16:57:14,919 - INFO - --- ЭТАП 1: СБОР HTML ДАННЫХ ---
2025-12-14 16:57:14,920 - INFO - Запуск краулинга с https://priem.stankin.ru/ до глубины 4...
2025-12-14 16:57:14,921 - INFO - -> Парсинг URL: https://priem.stankin.ru/ (Глубина: 0)
2025-12-14 16:57:15,539 - INFO - -> Парсинг URL: https://priem.stankin.ru/olympiads/ (Глубина: 1)
2025-12-14 16:57:16,148 - INFO - -> Парсинг URL: https://priem.stankin.ru/zadatvoprospk/ (Глубина: 1)
2025-12-14 16:57:16,713 - INFO - -> Парсинг URL: https://priem.stankin.ru/meropriyatiya/ (Глубина: 1)
2025-12-14 16:57:17,306 - INFO - -> Парсинг URL: https://priem.stankin.ru/rod_sobranie/ (Глубина: 1)
2025-12-14 16:57:17,874 - INFO - -> Парсинг URL: https://priem.stankin.ru/podkast_napravleniy/ (Глубина: 1)
2025-12-14 16:57:18,459 - INFO - -> Парсинг URL: https://priem.stankin.ru/_dod/ (Глубина: 1)
2025-12-14 16:57:19,051 - INFO - -> Парсинг URL: https://priem.stan

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2025-12-14 16:59:28,699 - INFO - Индексирован пакет: 0 - 500


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2025-12-14 16:59:35,387 - INFO - Индексирован пакет: 500 - 1000


...

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2025-12-14 16:59:44,870 - INFO - Индексирован пакет: 1500 - 2000


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

2025-12-14 16:59:46,963 - INFO - Индексирован пакет: 2000 - 2370
2025-12-14 16:59:46,987 - CRITICAL - SUCCESS: Индексирование завершено. Всего чанков в БД: 2370
2025-12-14 16:59:46,989 - CRITICAL - SUCCESS: Общее количество проиндексированных чанков: 2370


True

In [57]:
TEST_QUERY_1 = "Какие документы нужны для подачи заявления в МГТУ СТАНКИН?"
TEST_QUERY_2 = "Сколько баллов нужно набрать, для того чтобы поступить на направление 09.03.03?"
TEST_QUERY_3 = "Сколько баллов дают за индивидуальные достижения?"
TEST_QUERY_4 = "Какие направления подготовки бакалавриата есть в Станкине?"
TEST_QUERY_5 = "Когда начинаются вступительные испытания в магистратуре?"

test_rag_query(TEST_QUERY_1, ef)
test_rag_query(TEST_QUERY_2, ef)
test_rag_query(TEST_QUERY_3, ef)
test_rag_query(TEST_QUERY_4, ef)
test_rag_query(TEST_QUERY_5, ef)

2025-12-14 16:59:57,507 - INFO - --------------------------------------------------
2025-12-14 16:59:57,508 - INFO - НАЧАЛО ТЕСТИРОВАНИЯ КАЧЕСТВА RAG
2025-12-14 16:59:57,509 - INFO - Тестовый запрос: 'Какие документы нужны для подачи заявления в МГТУ СТАНКИН?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-14 16:59:57,580 - CRITICAL - 
--- 5 САМЫХ РЕЛЕВАНТНЫХ ЧАНКОВ (RAG-РЕЗУЛЬТАТ) ---

2025-12-14 16:59:57,582 - INFO - --------------------------------------------------
2025-12-14 16:59:57,582 - INFO - --------------------------------------------------
2025-12-14 16:59:57,583 - INFO - НАЧАЛО ТЕСТИРОВАНИЯ КАЧЕСТВА RAG
2025-12-14 16:59:57,583 - INFO - Тестовый запрос: 'Сколько баллов нужно набрать, для того чтобы поступить на направление 09.03.03?'


[1] Дистанция: 0.2862
    Источник: https://newcloud.stankin.ru/s/sndBbRQqy6wRLCz/download/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%BF%D1%80%D0%B8%D0%B5%D0%BC%D0%B0_2025_%D0%90%D0%A1%D0%9F.pdf
    Содержание: случае использования ЕПГУ для представления заявлений о приеме и документов, 
необходимых для поступления, МГТУ «СТАНКИН» вправе не проводить прием 
заявлений и документов посредством электронной информационной системы МГТУ 
«СТАНКИН». 
МГТУ «СТАНКИН» устанавливает места для приема заявлений и докум...

[2] Дистанция: 0.3070
    Источник: https://newcloud.stankin.ru/s/YozjQrwrYcGeDF3/download/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%BF%D1%80%D0%B8%D1%91%D0%BC%D0%B0_2025_%D0%91%D0%A1%D0%9C.pdf
    Содержание: ЕПГУ). 
МГТУ «СТАНКИН» обеспечивает возможность представления (направления) 
заявлений и документов, необходимых для поступления, всеми указанными способами. В 
случае использования ЕПГУ для представления заявлений о приеме и документов, 
необходимых для поступлени

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2025-12-14 16:59:57,720 - CRITICAL - 
--- 5 САМЫХ РЕЛЕВАНТНЫХ ЧАНКОВ (RAG-РЕЗУЛЬТАТ) ---

2025-12-14 16:59:57,721 - INFO - --------------------------------------------------
2025-12-14 16:59:57,722 - INFO - --------------------------------------------------
2025-12-14 16:59:57,723 - INFO - НАЧАЛО ТЕСТИРОВАНИЯ КАЧЕСТВА RAG
2025-12-14 16:59:57,723 - INFO - Тестовый запрос: 'Когда начинаются вступительные испытания в магистратуре?'


[1] Дистанция: 0.3491
    Источник: https://priem.stankin.ru/bakalavriatispetsialitet/info_perv_2025/
    Содержание: 1 курса бакалавриата и специалитета! Организационные собрания студентов 1 курса бакалавриата и специалитета будут проводиться в следующие сроки: --- НАЧАЛО ТАБЛИЦЫ --- | Unnamed: 0 | Unnamed: 1 | Unnamed: 2 | |-------------:|:---------------------------------------------------------------------|:---...

[2] Дистанция: 0.3575
    Источник: https://newcloud.stankin.ru/s/YozjQrwrYcGeDF3/download/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%BF%D1%80%D0%B8%D1%91%D0%BC%D0%B0_2025_%D0%91%D0%A1%D0%9C.pdf
    Содержание: настоящей главой. 
Сроки приема на обучение, которые не установлены настоящей главой, 
устанавливаются МГТУ «СТАНКИН» самостоятельно. 
130. При приеме на обучение на места в рамках контрольных цифр приема по 
программам бакалавриата и программам специалитета по всем формам обучения: 
1) прием заявле...

[3] Дистанция: 0.3630
    Источник: https://priem.stankin

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-14 16:59:57,765 - CRITICAL - 
--- 5 САМЫХ РЕЛЕВАНТНЫХ ЧАНКОВ (RAG-РЕЗУЛЬТАТ) ---

2025-12-14 16:59:57,767 - INFO - --------------------------------------------------


[1] Дистанция: 0.3041
    Источник: https://newcloud.stankin.ru/s/YozjQrwrYcGeDF3/download/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%BF%D1%80%D0%B8%D1%91%D0%BC%D0%B0_2025_%D0%91%D0%A1%D0%9C.pdf
    Содержание: необходимо сдавать внутренние вступительные испытания (далее – день завершения 
приема документов со сдачей вступительных испытаний);...

[2] Дистанция: 0.3637
    Источник: https://newcloud.stankin.ru/s/YozjQrwrYcGeDF3/download/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%BF%D1%80%D0%B8%D1%91%D0%BC%D0%B0_2025_%D0%91%D0%A1%D0%9C.pdf
    Содержание: − по убыванию количества баллов за отдельные вступительные испытания в 
соответствии 
с 
приоритетностью 
испытаний 
при 
ранжировании. 
Университет 
устанавливает 
следующую 
приоритетность 
вступительных 
испытаний, 
которая...

[3] Дистанция: 0.3652
    Источник: https://newcloud.stankin.ru/s/YozjQrwrYcGeDF3/download/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%BF%D1%80%D0%B8%D1%91%D0%BC%D0%B0_2025_%D0%91%D0%A1%D0%9C.pdf
  

In [59]:
test_rag_query("Что нужно сдавать для того чтобы поступить на направление 09.03.03", ef) 

2025-12-14 17:03:35,244 - INFO - --------------------------------------------------
2025-12-14 17:03:35,245 - INFO - НАЧАЛО ТЕСТИРОВАНИЯ КАЧЕСТВА RAG
2025-12-14 17:03:35,246 - INFO - Тестовый запрос: 'Что нужно сдавать для того чтобы поступить на направление 09.03.03'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-14 17:03:35,326 - CRITICAL - 
--- 5 САМЫХ РЕЛЕВАНТНЫХ ЧАНКОВ (RAG-РЕЗУЛЬТАТ) ---

2025-12-14 17:03:35,329 - INFO - --------------------------------------------------


[1] Дистанция: 0.3823
    Источник: https://priem.stankin.ru/bakalavriatispetsialitet/info_perv_2025/
    Содержание: до 18.00 ( для заселения в день получения ордера необходимо получить ордер с 9.00 до 14.00 , заселиться в общежитие можно только с понедельника по пятницу) при предъявлении паспорта. Единый деканат находится в здании по адресу Вадковский пер., д. 3а. При входе в здание по адресу Вадковский пер., д. ...

[2] Дистанция: 0.3959
    Источник: https://priem.stankin.ru/bakalavriatispetsialitet/info_perv_2025/
    Содержание: 2025 года | 09.03.02 и 15.05.01 | | nan | начиная с 28 августа 2025 года | 09.03.03, 15.03.01, 15.03.02, 27.03.01 | | nan | начиная с 29 августа 2025 года | 12.03.01, 15.03.05 | --- КОНЕЦ ТАБЛИЦЫ --- ВАЖНО: Сначала надо получить ордер, а только потом можно заселяться в общежитие Ордера выдаются в Ед...

[3] Дистанция: 0.3976
    Источник: https://newcloud.stankin.ru/s/sk5ermw7k9Kdxxs/download/2025_%D0%9F%D0%BE%D1%80%D1%8F%D0%B4%D0%BE%D0%BA_%D0%BF%D1%80%D0